## 코사인 유사도를 이용한 영화 추천 시스템

In [2]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [3]:
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

In [4]:
cos_sim(doc1,doc2), cos_sim(doc1,doc3), cos_sim(doc2, doc3)

(0.6666666666666667, 0.6666666666666667, 1.0000000000000002)

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([doc1],[doc2])

array([[0.66666667]])

In [6]:
cosine_similarity([doc2],[doc3])

array([[1.]])

In [7]:
import pandas as pd

In [19]:
movie = pd.read_csv('C:/Workspace/02.DataAnalysis/07.NLP/archive/movies_metadata.csv', low_memory=False)
movie.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [13]:
movie.shape

(45466, 24)

In [20]:
df = movie[['title','overview']]
df.head()

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


In [15]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
df.dropna(inplace=True)
df.shape

(44506, 2)

In [22]:
df.tail()

,title,overview
45461,Subdue,Rising and falling between a man and woman.
45462,Century of Birthing,An artist struggles to finish his work while a...
45463,Betrayal,"When one of her hits goes wrong, a professiona..."
45464,Satan Triumphant,"In a small town live two brothers, one a minis..."
45465,Queerama,50 years after decriminalisation of homosexual...


In [23]:
df.set_index('title',inplace=True)
df.reset_index(inplace=True)
df.tail()

,title,overview
44501,Subdue,Rising and falling between a man and woman.
44502,Century of Birthing,An artist struggles to finish his work while a...
44503,Betrayal,"When one of her hits goes wrong, a professiona..."
44504,Satan Triumphant,"In a small town live two brothers, one a minis..."
44505,Queerama,50 years after decriminalisation of homosexual...


### text 전처리

In [24]:
# 구둣점 제거
import re
df['clean_doc'] = df.overview.apply(lambda x: re.sub('[^A-Za-z ]','',x))
df.head()

,title,overview,clean_doc
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Led by Woody Andys toys live happily in his ro...
1,Jumanji,When siblings Judy and Peter discover an encha...,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Cheated on mistreated and stepped on the women...
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Just when George Banks has recovered from his ...


### DTM 변환

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(stop_words='english')
tfidf_matrix = tvect.fit_transform(df.overview)
tfidf_matrix